In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/items-orders/Orders data.xlsx
/kaggle/input/items-orders/Associated_Item_data.csv


In [2]:
dataset = pd.read_excel(r'/kaggle/input/items-orders/Orders data.xlsx')
associated_dataset = pd.read_csv(r'/kaggle/input/items-orders/Associated_Item_data.csv')

In [3]:
customer_behavior=pd.DataFrame()

In [4]:
customer_behavior['customer_id']=dataset['customer_id'].unique()
customer_behavior

,customer_id
0,5235
1,7622
2,3827
3,2223
4,6903
...,...
4156,1222
4157,7309
4158,7840
4159,2486


In [5]:
df=dataset.copy()

In [6]:
for idx in customer_behavior['customer_id']:
    customer_data = df[df['customer_id'] == idx]
    
    total_orders = customer_data['order_id'].nunique()
    total_order_days = customer_data['order_date'].nunique()
    
    average_orders_placed = total_orders / total_order_days if total_order_days > 0 else 0
    total_order_value = customer_data['net_order_amount'].sum()
    average_order_value = total_order_value / total_orders if total_orders > 0 else 0

    # Calculate average order gap days
    order_dates = customer_data['order_date'].sort_values()
    order_gaps = order_dates.diff().dt.days.dropna()
    avg_order_gap_days = order_gaps.mean() if not order_gaps.empty else 0

    # Order consistency score (1 / std of gaps)
    order_consistency_score = 1 / (order_gaps.std() + 1e-5) if len(order_gaps) > 1 else 0

    # Assign to customer_behavior DataFrame
    customer_behavior.loc[customer_behavior['customer_id'] == idx, 'Total Orders'] = total_orders
    customer_behavior.loc[customer_behavior['customer_id'] == idx, 'Total Order Days'] = total_order_days
    customer_behavior.loc[customer_behavior['customer_id'] == idx, 'Average Orders Placed'] = average_orders_placed
    customer_behavior.loc[customer_behavior['customer_id'] == idx, 'Total Order Value'] = total_order_value
    customer_behavior.loc[customer_behavior['customer_id'] == idx, 'Average Order Value'] = average_order_value
    customer_behavior.loc[customer_behavior['customer_id'] == idx, 'Average Order Gap Days'] = avg_order_gap_days
    customer_behavior.loc[customer_behavior['customer_id'] == idx, 'Order Consistency Score'] = order_consistency_score


In [7]:
customer_behavior.to_csv("Customer_Behavior_Data.csv",index=False)


In [8]:
customer_behavior

,customer_id,Total Orders,Total Order Days,Average Orders Placed,Total Order Value,Average Order Value,Average Order Gap Days,Order Consistency Score
0,5235,106.0,90.0,1.177778,1052898.0,9933.000000,1.790476,0.704750
1,7622,14.0,12.0,1.166667,48627.0,3473.357143,4.923077,0.197731
2,3827,8.0,8.0,1.000000,414750.0,51843.750000,18.714286,0.093425
3,2223,231.0,142.0,1.626761,1345798.0,5825.965368,0.830435,1.043287
4,6903,52.0,49.0,1.061224,89588.0,1722.846154,2.392157,0.492475
...,...,...,...,...,...,...,...,...
4156,1222,1.0,1.0,1.000000,41850.0,41850.000000,0.000000,0.000000
4157,7309,1.0,1.0,1.000000,7350.0,7350.000000,0.000000,0.000000
4158,7840,1.0,1.0,1.000000,16510.0,16510.000000,0.000000,0.000000
4159,2486,1.0,1.0,1.000000,2397.0,2397.000000,0.000000,0.000000
